#Importing libraries

In [1]:
# import libraries


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scipy

#importing data

In [5]:
path = r'/Users/office/Desktop/Instacart Basket Analysis'

In [6]:
path

'/Users/office/Desktop/Instacart Basket Analysis'

In [7]:
#import customers_data

df_customers = pd.read_csv(os.path.join(path,'02 Data','Original Data','customers.csv'),index_col = False)
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


#04. Data Wrangling

#Updating column data types

In [9]:
#change columns data type

df_customers['user_id'] = df_customers['user_id'].astype('str')
df_customers['date_joined'] = pd.to_datetime(df_customers['date_joined'])
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   user_id       206209 non-null  object        
 1   First Name    194950 non-null  object        
 2   Surnam        206209 non-null  object        
 3   Gender        206209 non-null  object        
 4   STATE         206209 non-null  object        
 5   Age           206209 non-null  int64         
 6   date_joined   206209 non-null  datetime64[ns]
 7   n_dependants  206209 non-null  int64         
 8   fam_status    206209 non-null  object        
 9   income        206209 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 15.7+ MB


#Updating column names

In [10]:
#rename first name, surnam, gender, state and age


df_customers.rename(columns={"First Name":"first name", "Surnam":"last name", "Gender":"gender", "STATE":"state",
                            "Age":"age"}, inplace=True)
df_customers.head()

,user_id,first name,last name,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,2017-01-01,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,2017-01-01,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,2017-01-01,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,2017-01-01,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,2017-01-01,1,married,40374


#05. Data consistency checks

In [11]:
#check for columns with mixed data type

for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

first name


In [12]:
#update 'first name' to string to data type


df_customers['first name'] = df_customers['first name'].astype('str')
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   user_id       206209 non-null  object        
 1   first name    206209 non-null  object        
 2   last name     206209 non-null  object        
 3   gender        206209 non-null  object        
 4   state         206209 non-null  object        
 5   age           206209 non-null  int64         
 6   date_joined   206209 non-null  datetime64[ns]
 7   n_dependants  206209 non-null  int64         
 8   fam_status    206209 non-null  object        
 9   income        206209 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 15.7+ MB


#Checking for missing values

In [13]:
#check for missing values

df_customers.isnull().sum()

user_id         0
first name      0
last name       0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

#There are no missing values

#checking for duplicates

In [14]:
#check for duplicates

df_dups = df_customers[df_customers.duplicated()]
df_dups

,user_id,first name,last name,gender,state,age,date_joined,n_dependants,fam_status,income


#there are no duplicates

#06. combine orders_products with customers data

In [15]:
#import orders_products_data

df_ords_prods = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','ords_prods_merged_updated_2.pkl'))
df_ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434212 entries, 0 to 32434211
Data columns (total 24 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  mean_spend              float64 
 21  spende

#changing column data type for the user_id in orders_products_dataframe

In [16]:
#change column data type

df_ords_prods['user_id'] = df_ords_prods['user_id'].astype('str')

In [17]:
df_ords_prods['user_id'].dtype

dtype('O')

#dropping unwanted column from the orders_products dataframe

In [18]:
#drop the '_merge' column 

df_ords_prods.drop(columns=['_merge'], inplace=True)
df_ords_prods.shape

(32434212, 23)

In [19]:
# merge ords_prods with customers using user_id


df_ords_prods_customers_left = df_ords_prods.merge(df_customers, on=['user_id'], indicator=True)

In [20]:
df_ords_prods_customers_left['_merge'].value_counts(dropna=False)

_merge
both          32434212
left_only            0
right_only           0
Name: count, dtype: int64

In [21]:
df_ords_prods_customers_left.shape

(32434212, 33)

In [23]:
df_ords_prods_customers_left.drop(columns=['_merge'], inplace=True)
df_ords_prods_customers_left.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434212 entries, 0 to 32434211
Data columns (total 32 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   order_id                int64         
 1   user_id                 object        
 2   order_number            int64         
 3   orders_day_of_week      int64         
 4   order_hour_of_day       int64         
 5   days_since_prior_order  float64       
 6   product_id              int64         
 7   add_to_cart_order       int64         
 8   reordered               int64         
 9   product_name            object        
 10  aisle_id                int64         
 11  department_id           int64         
 12  prices                  float64       
 13  price_range_loc         object        
 14  busiest day             object        
 15  busiest_days            object        
 16  busiest_period_of_day   object        
 17  max_order               int64         
 18  

#Exporting merged customer file

In [22]:
#Export merged ccustomer data

df_ords_prods_customers_left.to_pickle(os.path.join(path,'02 Data','Prepared Data','ords_prods_customers_left.pkl'))